# CS 179 Project

In [3]:
import numpy as np
import pyGM as gm

# names of 500 movies to be rated:
with open('top-names.txt') as f: names = f.read().split('\n')
    
# ratings = int(2000 x 500) ratings of 500 movies by 2000 people; -1 = not rated
ratings = np.loadtxt('top-ratings-missing.txt')
nUsers,nMovies = ratings.shape

X = (ratings >= 7).astype(int);   # did each user like the movie?  (binary)
# (use any threshold you like, but "7+" might be "worth recommending"?)

# Let's split into training & test:
np.random.seed(0)
pi = np.random.permutation(nUsers)
iTr,iTe = pi[:int(nUsers*.7)], pi[int(nUsers*.7):]
Xtr,Xte = X[iTr,:],X[iTe,:]

In [4]:
# for a prediction task, censor some entries in X[iTe,:]:
Xte_missing = np.copy(Xte)
Xte_missing[ np.random.random_sample(Xte.shape)<=.3 ] = -1  # discard 30% of observations

# Now, we can measure the likelihood of Xte,
# or predict Xte given Xte_missing

We can compare our model's accuracy/ improvement by comparing Xtr's and Xte's LL value when calculated independently vs an Ising model.

### Calculating Independently:

In [5]:
pXi = np.mean(1.*Xtr,axis=0);  # empirical probability of liking each movie individually
model0 = gm.GraphModel( [gm.Factor([gm.Var(i,2)],[1-pXi[i],pXi[i]]) for i in range(nMovies)])

# How well does our model fit the data?
print("Independent model Train LL: ",np.mean([model0.logValue(x) for x in Xtr]))
print("Independent model Test  LL: ",np.mean([model0.logValue(x) for x in Xte]))

Independent model Train LL:  -284.862769233602
Independent model Test  LL:  -285.486137585076


### Calculating using Ising model:

#### Ising learning algorithm: Find adjacency graph using penalized logistic regression

In [6]:
from sklearn.linear_model import LogisticRegression

# for each Xi, estimate the neighborhood of Xi using L1-reg logistic regression:
C = 1./100  # inverse regularization penalty: smaller => sparser weights
nbrs,th_ij,th_i = [None]*nMovies, [None]*nMovies, np.zeros((nMovies,))
Xtmp = np.copy(Xtr)  # we'll be messing with the matrix
for i in range(nMovies):  
    Xtmp[:,i] = 0.      # remove ourselves
    lr = LogisticRegression(penalty='l1',C=C,solver='liblinear').fit(Xtmp,Xtr[:,i])
    nbrs[i] = np.where(np.abs(lr.coef_) > 1e-6)[1]
    th_ij[i]= lr.coef_[0,nbrs[i]]/2.
    th_i[i] = lr.intercept_/2.
    Xtmp[:,i] = Xtr[:,i]; # & restore after

In [7]:
print("Average connectivity at C=",C,": ")
print(np.mean([len(nn) for nn in nbrs])," +/- ",np.std([len(nn) for nn in nbrs]))

Average connectivity at C= 0.01 : 
1.096  +/-  2.137939194645161


In [8]:
# Collect single-variable factors
factors1 = [gm.Factor(gm.Var(i,2),[-t,t]).exp() for i,t in enumerate(th_i)]

# Collect non-zero pairwise factors
for i in range(nMovies):
    for j,n in enumerate(nbrs[i]):
        scope = [gm.Var(i,2),gm.Var(int(n),2)]
        t = th_ij[i][j]
        factors1.append( gm.Factor(scope, [[t,-t],[-t,t]]).exp() )

# Build a model from the factors
model1 = gm.GraphModel(factors1)
model1.makeMinimal()

In [9]:
# How well does our model fit the data?  Now we need to know the partition function!
# Maybe we can do it using variable elimination?  Depends on the treewidth.
order,val = gm.eliminationOrder(model1, 'minfill')
# (actually "val" here is an estimate of the required memory, so that's even better, but...)

pt = gm.PseudoTree(model1,order)                   # find the tree width:
print("Induced width of order found: ",pt.width)

Induced width of order found:  16


In [10]:
# 16 is not too bad (2^16 = 65k), so we can do it:
import pyGM.wmb
jt = gm.wmb.JTree(model1, elimOrder=order)
lnZ = jt.msgForward()
print("Log partition f'n: ",lnZ)

Log partition f'n:  396.02023094234


Log-likelihood is slightly better than the independent model.

In [11]:
# Now we can calculate the actual log-likelihood of the data:
print("LogReg Ising Train LL: ",np.mean([model1.logValue(x) for x in Xtr]) - lnZ)
print("LogReg Ising Test  LL: ",np.mean([model1.logValue(x) for x in Xte]) - lnZ)

LogReg Ising Train LL:  -281.4238762978957
LogReg Ising Test  LL:  -282.77572242876647


#### Ising learning algorithm: Maximum likelihood parameters given adjecency graph

#### Ising learning algorithm: Maximum pseudolikelihood params given adjacency graph

#### Ising learning algorithm: Find both together using multiplicative weights

### Evaluating model quality:

In [12]:
def conditional(factor,i,x):
    return factor.t[tuple(x[v] if v!=i else slice(v.states) for v in factor.vars)]

def pseudolikelihood(model,X):
    LL = np.zeros( X.shape )
    for i in range(X.shape[1]):  # for each variable:
        flist = model.factorsWith(i, copy=False)
        for j in range(X.shape[0]):
            pXi = 1.
            for f in flist: pXi *= conditional(f,i,X[j])
            LL[j,i] = np.log( pXi[X[j,i]]/pXi.sum() ); 
    return LL.sum(1);

Evaluate each model, including the ones we make ourselves.

In [13]:
pseudolikelihood(model0, Xte).mean()

-285.4861375850758

In [14]:
pseudolikelihood(model1, Xte).mean()

-274.69567095774784

### Making predictions given the Ising model parameters:

In [16]:
def impute_missing(model, Xobs):
    m,n = Xobs.shape
    Xhat = np.copy(Xobs);
    for j in range(m):
        x_obs = {i:Xobs[j,i] for i in range(n) if Xobs[j,i] >= 0}
        x_unobs = [i for i in range(n) if Xobs[j,i] < 0]
        cond = gm.GraphModel([f.condition(x_obs) for f in model.factorsWithAny(x_unobs)])
        for x in cond.X:
            if x.states == 0: x.states = 1;  # fix a bug in GraphModel behavior for missing vars...
        jt = gm.wmb.JTree(cond, weights=1e-6) # 0: for maximization
        x_hat = jt.argmax();
        for i in x_unobs: Xhat[j,i] = x_hat[i]
    return Xhat

In [17]:
# Slow!  (Constructing lots of conditional models...)
Xte_hat = impute_missing(model1, Xte_missing)
print('Error rate:', np.mean( (Xte_hat!=Xte_missing)[Xte_hat<0] ) )

Error rate: nan


C:\Users\yerli\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\yerli\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
